# DAE solver

In this notebook, we show some examples of solving a DAE model. For the purposes of this example, we use the CasADi solver, but the syntax remains the same for other solvers

In [ ]:
%pip install "pybamm[plot,cite]" -q    # install PyBaMM if it is not installed
import pybamm
import numpy as np
import os
import matplotlib.pyplot as plt

os.chdir(pybamm.__path__[0] + "/..")

## Integrating DAEs

In PyBaMM, a model is solved by calling a solver with solve. This sets up the model to be solved, and then calls the method `_integrate`, which is specific to each solver. We begin by setting up and discretising a model

In [ ]:
# Create model
model = pybamm.BaseModel()
u = pybamm.Variable("u")
v = pybamm.Variable("v")
model.rhs = {u: -v}  # du/dt = -v
model.algebraic = {v: 2 * u - v}  # 2*v = u
model.initial_conditions = {u: 1, v: 2}
model.variables = {"u": u, "v": v}

# Discretise using default discretisation
disc = pybamm.Discretisation()
disc.process_model(model);

Now the model can be solved by calling `solver.solve` with a specific time vector at which to evaluate the solution

In [ ]:
# Solve #################################
t_eval = np.linspace(0, 2, 30)
dae_solver = pybamm.CasadiSolver(mode="safe")
solution = dae_solver.solve(model, t_eval)
#########################################

# Extract u and v
t_sol = solution.t
u = solution["u"]
v = solution["v"]

# Plot
t_fine = np.linspace(0, t_eval[-1], 1000)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(13, 4))
ax1.plot(t_fine, np.exp(-2 * t_fine), t_sol, u(t_sol), "o")
ax1.set_xlabel("t")
ax1.legend(["exp(-2*t)", "u"], loc="best")

ax2.plot(t_fine, 2 * np.exp(-2 * t_fine), t_sol, v(t_sol), "o")
ax2.set_xlabel("t")
ax2.legend(["2*exp(-2*t)", "v"], loc="best")

plt.tight_layout()
plt.show()

Note that, where possible, the solver makes use of the mass matrix and jacobian for the model. However, the discretisation or solver will have created the mass matrix and jacobian algorithmically, using the expression tree, so we do not need to calculate and input these manually.

The solution terminates at the final simulation time:

In [ ]:
solution.termination

### Events

It is possible to specify events at which a solution should terminate. This is done by adding events to the `model.events` dictionary. In the following example, we solve the same model as before but add a termination event when `v=-0.2`.

In [ ]:
# Create model
model = pybamm.BaseModel()
u = pybamm.Variable("u")
v = pybamm.Variable("v")
model.rhs = {u: -v}  # du/dt = -v
model.algebraic = {v: 2 * u - v}  # 2*v = u
model.initial_conditions = {u: 1, v: 2}
model.events.append(pybamm.Event("v=0.2", v - 0.2))  # adding event here

model.variables = {"u": u, "v": v}

# Discretise using default discretisation
disc = pybamm.Discretisation()
disc.process_model(model)

# Solve #################################
t_eval = np.linspace(0, 2, 30)
dae_solver = pybamm.CasadiSolver(mode="safe")
solution = dae_solver.solve(model, t_eval)
#########################################

# Extract u and v
t_sol = solution.t
u = solution["u"]
v = solution["v"]

# Plot
t_fine = np.linspace(0, t_eval[-1], 1000)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(13, 4))
ax1.plot(t_fine, np.exp(-2 * t_fine), t_sol, u(t_sol), "o")
ax1.set_xlabel("t")
ax1.legend(["exp(-2*t)", "u"], loc="best")

ax2.plot(
    t_fine,
    2 * np.exp(-2 * t_fine),
    t_sol,
    v(t_sol),
    "o",
    t_fine,
    0.2 * np.ones_like(t_fine),
    "k",
)
ax2.set_xlabel("t")
ax2.legend(["2*exp(-2*t)", "v", "v = 0.2"], loc="best")

plt.tight_layout()
plt.show()

Now the solution terminates because the event has been reached

In [ ]:
solution.termination

## Finding consistent initialization

The solver will fail if initial conditions that are inconsistent with the algebraic equations are provided. However, before solving the DAE solvers automatically use `_set_consistent_initialization` to obtain consistent initial conditions, starting from a guess of bad initial conditions, using a simple root-finding algorithm. 

In [ ]:
# Create model
model = pybamm.BaseModel()
u = pybamm.Variable("u")
v = pybamm.Variable("v")
model.rhs = {u: -v}  # du/dt = -v
model.algebraic = {v: 2 * u - v}  # 2*v = u
model.initial_conditions = {u: 1, v: 1}  # bad initial conditions, solver fixes
model.events.append(pybamm.Event("v=0.2", v - 0.2))
model.variables = {"u": u, "v": v}

# Discretise using default discretisation
disc = pybamm.Discretisation()
disc.process_model(model)
print(f"y0_guess={model.concatenated_initial_conditions.evaluate().flatten()}")

dae_solver = pybamm.CasadiSolver(mode="safe")
dae_solver.set_up(model)
dae_solver._set_consistent_initialization(model, 0, {})
print(f"y0_fixed={model.y0}")

## References

The relevant papers for this notebook are:

In [ ]:
pybamm.print_citations()